In [2]:
import pandas as pd 
import requests 
from bs4 import BeautifulSoup 

Scrap from the URL and pull the table

In [3]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") 
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]  
df_temp = pd.read_html(str(table)) 
neighborhood=pd.DataFrame(df_temp[0]) 


In [4]:
neighborhood.shape

(20, 9)

Converting data frame to list to parse each cell and create new data frame in the required format i.e. three columns postal code, borough and nieghbourhood.

In [6]:
neighbourhood_list = [neighborhood.columns.values.tolist()] + neighborhood.values.tolist()

following steps are done in below code block:
  a. Create an empty dataframe with required columns

  b. Iterate though list

  c. Ignore the list where "Not assinged" is found

  d. Slice first 3 characters as Postal code

  e. Find index position of '(' & ')'

  f. We have assumed that only one open round and close bracket exists

  g. Then used substring to pull string between brackerts and replaced '/' with comma and replaced additonal space as well

  h. Appended each record to data frame

  i. it is assume assumed that each cell with have neighbourhood and borough


In [7]:
df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])
print(df)
for n_list in neighbourhood_list:
  for neighbour in n_list:
    neighbour=str(neighbour)
    if (neighbour.find('Not assigned') == -1):
      if len(neighbour)>3:
        postal_code=neighbour[:3]
        start=neighbour.find('(', 1)
        end=neighbour.find(')', 1)
        neighd = neighbour[start+1:end].replace('/',',').replace(' ,',',')
        borough=neighbour[3:start]
        df = df.append({'PostalCode' : postal_code, 'Borough' : borough, 'Neighborhood' : neighd},ignore_index=True)

        

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []


In [22]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [82]:
df.to_csv('file1.csv')

In [83]:
df.shape

(103, 3)

In [84]:
len(df['PostalCode'].unique())

103

**Below step are attempted to fetch geo co-ordinate using geocoder module**

In [10]:
pip install geocoder

     |████████████████████████████████| 102kB 4.2MB/s 


**Beow code is used to find latitude & longitude using address**

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
g = geocoder.google('Parkwoods, North York, M3A')
lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

**The code kept failing and hence given csv file containing geo code was uploaded and it will be merged to original data frame containing PostalCode, Borough & Neighbourhood**

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
# Step 1: Load instance usage data from instanceUsage_data.csv to pandas data frame 
# import libraries
import pandas as pd
import numpy as np
# needed path for csv can be specified
df_geo_code = pd.read_csv('/content/Geospatial_Coordinates.csv')


In [27]:
df_geo_code

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [26]:
df_geo_code.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)

**Below code help to migrate two data frames using PostalCode as key**

In [30]:
result = pd.merge( df,df_geo_code,on=('PostalCode') ,how='inner')

In [31]:
result

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
